In [1]:
# libraries needed
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation,Flatten,Dropout
from tensorflow.keras.layers import Conv2D,MaxPooling2D, AveragePooling2D

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

from matplotlib import pyplot as plt

## Data Augmentation

In [2]:
train = ImageDataGenerator(rotation_range=20,
                            zoom_range=0.15,
                            width_shift_range=0.2,
                            height_shift_range=0.2,
                            shear_range=0.15,
                            horizontal_flip=True,
                            validation_split=0.2)

In [3]:
train_path = 'Dataset/'

In [4]:
training_data = train.flow_from_directory(train_path,
                                          target_size=(224, 224),
                                          class_mode='categorical',
                                          subset="training")

Found 22048 images belonging to 2 classes.


In [5]:
validation_data = train.flow_from_directory(train_path,
                                          target_size=(224, 224),
                                          class_mode='categorical',                                          
                                          subset="validation")

Found 5510 images belonging to 2 classes.


In [6]:
training_data.class_indices

{'Parasitized': 0, 'Uninfected': 1}

## Transfer Learning

In [7]:
# load the MobileNetV2 network, ensuring the head FC layer sets are left off
Head = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

In [9]:
for layer in Head.layers:
    layer.trainable = False

In [10]:
base = Sequential()

In [11]:
base.add(
    Head
)

base.add(
    AveragePooling2D(pool_size=(7,7))
)

base.add(
    Flatten()
)

base.add(
    Dense(128, activation="relu")
)

base.add(
    Dropout(0.5)
)

base.add(
    Dense(2, activation="softmax")
)

In [13]:
#base.summary()

In [15]:
base.compile(loss="categorical_crossentropy", 
              optimizer="adam",
              metrics=["accuracy"])

In [16]:
history = base.fit_generator(training_data,
                             epochs=10,
                             validation_data=validation_data)

/opt/anaconda3/envs/akshit_venv/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
689/689 [==============================] - 1091s 2s/step - loss: 0.6873 - accuracy: 0.6321 - val_loss: 0.5135 - val_accuracy: 0.7439
Epoch 2/10
689/689 [==============================] - 1045s 2s/step - loss: 0.5277 - accuracy: 0.7368 - val_loss: 0.4868 - val_accuracy: 0.7713
Epoch 3/10
689/689 [==============================] - 1042s 2s/step - loss: 0.5066 - accuracy: 0.7523 - val_loss: 0.4678 - val_accuracy: 0.7780
Epoch 4/10
689/689 [==============================] - 1047s 2s/step - loss: 0.4946 - accuracy: 0.7542 - val_loss: 0.4606 - val_accuracy: 0.7735
Epoch 5/10
689/689 [==============================] - 1052s 2s/step - loss: 0.4880 - accuracy: 0.7588 - val_loss: 0.4500 - val_accuracy: 0.7864
Epoch 6/10
689/689 [==============================] - 1075s 2s/step - loss: 0.4874 - accuracy: 0.7608 - val_loss: 0.4642 - val_accuracy: 0.7927
Epoch 7/10
689/689 [==============================] - 1050s 2s/step - loss: 0.4826 - accuracy: 0.7688 - val_loss: 0.4434 - val_accuracy: